##Imports

In [ ]:
import csv as csv 
import numpy as np
import pandas as pd
import pylab as P
import sklearn

In [ ]:
%pylab inline

##Read the Data Files

In [ ]:
train = pd.read_csv('data/train.csv', header=0)
train_size = len(train.index)
train_size

In [ ]:
train.tail(3)

In [ ]:
train.dtypes

In [ ]:
train.info()

In [ ]:
train.describe()

In [ ]:
test = pd.read_csv('data/test.csv', header=0)
test_size = len(test.index)
test_size

In [ ]:
test.tail(3)

In [ ]:
d = train.append(test)
d.index = pd.Index(np.arange(len(d.index)))
d.tail(3)

##Data Munging

In [ ]:
d['Age'][0:10]

In [ ]:
d['Age'].mean()

In [ ]:
d[d['Age'] > 60][['Sex', 'Pclass', 'Age', 'Survived']]

In [ ]:
for i in range(1,4):
    print i, len(d[ (d['Sex'] == 'male') & (d['Pclass'] == i) ])

In [ ]:
d['Age'].hist()
P.show()

In [ ]:
d['Age'].dropna().hist(bins=16, range=(0,80), alpha = .5)
P.show()

##Data Cleaning

In [ ]:
pd.value_counts(d['Sex'])

In [ ]:
d['Gender'] = d['Sex'].map( {'female': 0, 'male': 1} ).astype(int)

In [ ]:
pd.value_counts(d['Embarked'])

In [ ]:
d['Embarked'].fillna('X', inplace=True)
d['Station'] = d['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2, 'X': 3} , ).astype(int)

In [ ]:
median_ages = np.zeros((2,3))
for i in range(0, 2):
    for j in range(0, 3):
        median_ages[i,j] = d[(d['Gender'] == i) & (d['Pclass'] == j+1)]['Age'].dropna().median()
median_ages

In [ ]:
d['AgeFill'] = d['Age']
for i in range(0, 2):
    for j in range(0, 3):
        d.loc[ (d.Age.isnull()) & (d.Gender == i) & (d.Pclass == j+1), 'AgeFill'] = median_ages[i,j]

In [ ]:
d[ d['Age'].isnull() ][['Gender','Pclass','Age','AgeFill']].head(10)

In [ ]:
d['AgeIsNull'] = pd.isnull(d.Age).astype(int)

In [ ]:
 d.describe()

##Feature Engineering

In [ ]:
d['FamilySize'] = d['SibSp'] + d['Parch']

In [ ]:
d['AgeXClass'] = d.AgeFill * d.Pclass

##Final Preparation

In [ ]:
d.dtypes

In [ ]:
d.dtypes[d.dtypes.map(lambda x: x=='object')]

In [ ]:
d = d.drop(['Age', 'Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'PassengerId'], axis=1)

In [ ]:
cols = d.columns.difference(['Survived']).tolist()
cols = ['Survived'] + cols
cols
d = d[cols]

In [ ]:
d.info()

In [ ]:
train = d.ix[:train_size-1, :]
train.tail(3)

In [ ]:
test = d.ix[train_size:, :]
test.tail(3)

In [ ]:
train = train.dropna()

In [ ]:
train_values = train.values
train_values

##Predict

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
forest = RandomForestClassifier(n_estimators = 100)
forest = forest.fit(train_values[0::,1::],train_values[0::,0])

In [ ]:
test.Survived = 0
test = test.dropna()
test_values = test.values.copy()
output = forest.predict(test_values[0::,1::])
output